In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
pd.set_option('display.max_colwidth', 100)

In [ ]:
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from evaluation_metrics import compute_metrics, compute_metrics_csv, mean_absolute_percentage_error, symetric_mean_absolute_percentage_error

In [ ]:
from statsmodels.tsa.stattools import acf, pacf, ccf, ccovf
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.stats import boxcox, yeojohnson
from scipy.spatial.distance import cosine, euclidean
import rstl

In [ ]:
from scipy.stats import boxcox, yeojohnson

In [ ]:
from tqdm.notebook import trange, tqdm
from datetime import datetime
import itertools
from numpy.polynomial import Polynomial as P

In [ ]:
# from lightgbm import LGBMRegressor

In [ ]:
import pickle
import calendar

In [ ]:
import ruptures as rpt
# from dtaidistance import dtw

In [ ]:
from river import tree

In [ ]:
def plot_pred_bkps(data, my_bkps, from_dt=None, title=None):
    if from_dt:
        data = data[data.index >= from_dt]
    my_bkps = [x for x in my_bkps if x < len(data)]
    
    df_err = data.copy()
    df_err.loc[:, 'AE'] = (df_err.y_true - df_err.y_pred).abs()
    df_err.loc[:, 'SE'] = (df_err.y_true - df_err.y_pred) ** 2
    df_err.loc[:, 'SAPE'] = ((df_err.y_pred - df_err.y_true).abs() / ((df_err.y_true.abs() + df_err.y_pred.abs())/2)) * 100
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data.y_true,
                            mode='lines',
                            line_color='#333C83', name='y_true'))
    
    fig.add_trace(go.Scatter(x=data.index, y=data.y_pred,
                            mode='lines',
                            line_color='red', name='y_pred'))
    clr_selection = 'green'
    color_switch = lambda x: 'blue' if x != 'blue' else 'green'
    
    for idx, cp in enumerate(my_bkps):
        if cp >= len(data):
            break

        fig.add_vline(x=data.index[cp], line_width=3, line_dash="dash", line_color="red")

        if idx < len(my_bkps) - 2:
            clr_selection = color_switch(clr_selection)
            fig.add_vrect(x0=data.index[cp], x1=data.index[my_bkps[idx+1]], line_width=0, fillcolor=clr_selection, opacity=0.1)
    
    fig.update_layout(title=title)
    fig.show()
    
    for roll in [24, 24*7]:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data.index, y=df_err['SAPE'].rolling(roll).mean(),
                                mode='lines',
                                line_color='#333C83', name=f'SAPE rolling({roll})'))

        for idx, cp in enumerate(my_bkps):
                if cp >= len(data):
                    break

                fig.add_vline(x=data.index[cp], line_width=3, line_dash="dash", line_color="red")

        fig.update_layout(title=f'SAPE rolling({roll})')
        fig.show()
    
    for x in reversed(['AE', 'SE', 'SAPE']):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data.index, y=df_err[x],
                                mode='lines',
                                line_color='#333C83', name=x))

        for idx, cp in enumerate(my_bkps):
            if cp >= len(data):
                break

            fig.add_vline(x=data.index[cp], line_width=3, line_dash="dash", line_color="red")
        fig.update_layout(title=x)
        fig.show()

In [ ]:
# plot_pred_bkps(df_forecast_res, my_bkps, '2013-01-05 00:00:00')

In [ ]:
# plot_pred_bkps(df_forecast_res, my_bkps, '2014-01-01 00:00:00')

In [ ]:
# pd.set_option('display.precision', 5)
# pd.set_option('display.float_format',  '{:.3g}'.format)

In [ ]:
results = {
    'SMCA': 'Results/NGC_HT_Day2Day_Online_2021.csv',
    'QDMDC': 'Results/NGC_HT_Day2Day_BySeason_2021.csv',
    'PCPDMC (Low)': 'Results/NGC_HT_Day2Day_ByCP_LOW_2021.csv',
    'PCPDMC (Medium)': 'Results/NGC_HT_Day2Day_ByCP_MED_2021.csv',
    'PCPDMC (High)': 'Results/NGC_HT_Day2Day_ByCP_HIGH_2021.csv',
    'MCPDMC (Low,WAVG)': 'Results/NGC_HT_Day2Day_ByCP_LOW_ERR_WAVG_2021.csv',
    'MCPDMC (Medium,WAVG)': 'Results/NGC_HT_Day2Day_ByCP_MED_ERR_WAVG_2021.csv',
    'MCPDMC (High,WAVG)': 'Results/NGC_HT_Day2Day_ByCP_HIGH_ERR_WAVG_2021.csv',
    'MCPDMC (Low,SWITCH)': 'Results/NGC_HT_Day2Day_ByCP_LOW_ERR_Switch_2021.csv',
    'MCPDMC (Medium,SWITCH)': 'Results/NGC_HT_Day2Day_ByCP_MED_ERR_Switch_2021.csv',
    'MCPDMC (High,SWITCH)': 'Results/NGC_HT_Day2Day_ByCP_HIGH_ERR_Switch_2021.csv'
}

In [ ]:
results = {k: pd.read_csv(v, sep=',', index_col=0) for k,v in results.items()}

In [ ]:
global_metrics = {k: compute_metrics(v[v.index >= '2014-01-05 00:00:00'].dropna()).loc[:, ['MAE', 'MSE', 'SMAPE']] for k,v in results.items()}

In [ ]:
from decimal import Decimal
pd.set_option('display.precision', 3)
pd.set_option('display.float_format',  '{:.4g}'.format)
# pd.set_option('display.float_format',  None)

In [ ]:
global_metrics['SMCA']

In [ ]:
list(global_metrics.keys())

In [ ]:
df_global = pd.DataFrame(columns=global_metrics['SMCA'].columns, index=list(global_metrics.keys()))
for k,v in global_metrics.items():
    df_global.loc[k, :] = v.values

In [ ]:
df_global

In [ ]:
global_metrics['SMCA'].MAE.apply(lambda x: f"{Decimal(x):.3g}")

In [ ]:
def year_metrics(df_forecast_res):
    df_pred_all = pd.DataFrame(columns=['MAE', 'MSE', 'SMAPE'], index=[x for x in range(2014, 2021)])

    for year in range(2014, 2021):
        tmp = compute_metrics(df_forecast_res[(df_forecast_res.index >= f'{year}-01-01 00:00:00') & (df_forecast_res.index <= f'{year}-12-31 00:00:00')].dropna()).loc[:, ['MAE', 'MSE', 'SMAPE']].values
        df_pred_all.loc[year, :] = tmp
    return df_pred_all

In [ ]:
yearly_metrics = {k: year_metrics(v) for k,v in results.items()}

In [ ]:
yearly_metrics['SMCA']

In [ ]:
df_detail = pd.DataFrame(columns=global_metrics['SMCA'].columns)
for k,v in yearly_metrics.items():
    df_detail = pd.concat([df_detail, pd.DataFrame(index=[k], columns=global_metrics['SMCA'].columns)])
    df_detail = pd.concat([df_detail, v])

In [ ]:
skip = 0
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 1
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 2
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 3
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 4
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 5
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 6
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 7
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 8
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 9
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
skip = 10
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
df_detail

In [ ]:
yearly_metrics.keys()

In [ ]:
fig = go.Figure()

mode = 'Low'
metric = 'SMAPE'
keys = [x for x in yearly_metrics.keys() if mode in x or 'SMCA' in x or 'QDMDC' in x]
years = yearly_metrics['SMCA'].index

for k in keys:
    data = yearly_metrics[k]
    name = k.replace(f'({mode},', '').replace(f'({mode})', '').replace(f' WAVG)', '-WA').replace(f' SWITCH)', '-SW')
    fig.add_trace(go.Scatter(x=years, y=data[metric],
                        mode='lines+markers',
                        name=name))
fig.update_layout(showlegend=True, xaxis_title='Year', yaxis_title=f'{metric} (%)', margin=dict(l=0, r=0, t=0, b=0), legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        size=22,
        # color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
y_true = results['PCPDMC (Low)'].loc[results['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_true
y_pred_PCPDMC = results['PCPDMC (Low)'].loc[results['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_WA = results['MCPDMC (High,WAVG)'].loc[results['MCPDMC (High,WAVG)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_SW = results['MCPDMC (High,SWITCH)'].loc[results['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred

y_pred_MCPDMC_SW

In [ ]:
bkps_low = pd.read_csv('Results/BKPS_LOW.csv')
bkps_low = bkps_low.iloc[:4]
bkps_low.DateTime = bkps_low.DateTime.apply(lambda x: x.replace('2013', '2020'))
bkps_low

In [ ]:
bkps_high = pd.read_csv('Results/BKPS_HIGH.csv')
bkps_high = bkps_high.iloc[:13]
bkps_high.DateTime = bkps_high.DateTime.apply(lambda x: x.replace('2013', '2020'))
bkps_high

In [ ]:
fig = go.Figure()

plot_data_kw = {'PCPDMC (Low)': y_pred_PCPDMC, 'MCPDMC-WA (High)': y_pred_MCPDMC_WA,'MCPDMC-SW (High)':y_pred_MCPDMC_SW , 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Gas Consumption', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
))
fig.show()

In [ ]:
fig = go.Figure()

plot_data_kw = {'PCPDMC (Low)': y_pred_PCPDMC, 'MCPDMC-WA (High)': y_pred_MCPDMC_WA,'MCPDMC-SW (High)':y_pred_MCPDMC_SW , 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v.loc[(v.index >= '2020-03-26 00:00:00') & (v.index <= '2020-04-20 00:00:00')]
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Gas Consumption', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
y_true = results['PCPDMC (Low)'].loc[results['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_true

y_pred = results['PCPDMC (Low)'].loc[results['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_pred
smape_PCPDMC = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results['MCPDMC (High,WAVG)'].loc[results['MCPDMC (High,WAVG)'].index >= '2020-01-01 00:00:00'].y_pred
smape_MCPDMC_WA = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results['MCPDMC (High,SWITCH)'].loc[results['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
smape_MCPDMC_SW = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred_MCPDMC_SW

In [ ]:
df_sape = pd.DataFrame(index=y_true.index)
df_sape['PCPDMC (Low)'] = smape_PCPDMC
df_sape['MCPDMC-WA (High)'] = smape_MCPDMC_WA
df_sape['MCPDMC-SW (High)'] = smape_MCPDMC_SW
df_sape['Month'] = pd.DatetimeIndex(df_sape.index).month
df_sape

In [ ]:
fig = go.Figure()

plot_data_kw = {'PCPDMC (Low)': smape_PCPDMC, 'MCPDMC-WA (High)': smape_MCPDMC_WA, 'MCPDMC-SW (High)':smape_MCPDMC_SW}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='SAPE (%)', margin=dict(l=0, r=0, t=0, b=0), 
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
df_sape['Month']

In [ ]:
df_sape_s = df_sape.iloc[:, :3].stack().reset_index(name='SAPE').rename({'level_1': 'Approach'}, axis=1)
df_sape_s['Month'] = pd.DatetimeIndex(df_sape_s['level_0']).month
df_sape_s

In [ ]:
np.arange(1, 13)

In [ ]:
[calendar.month_name[x] for x in np.arange(1, 13)]

In [ ]:
fig = px.box(df_sape_s, x='Month', y='SAPE', color='Approach')
fig.update_layout(showlegend=True, xaxis_title='Month', yaxis_title='SAPE (%)', margin=dict(l=0, r=0, t=0, b=0), legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
),
font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ),                 
xaxis = dict(
tickmode = 'array',
tickvals = np.arange(1, 13),
ticktext = [calendar.month_name[x] for x in np.arange(1, 13)]
))
fig.show()

In [ ]:
df_sape_agg = df_sape.groupby('Month').median()
df_sape_agg.index = [calendar.month_name[x] for x in df_sape_agg.index]
df_sape_agg